In [9]:
train_annotation = "annotation_train.odgt"
val_annotation = "annotation_val.odgt"
import cv2

In [16]:
from collections import defaultdict
import json
import os
with open(train_annotation, "r") as f:
    train_data = f.readlines()
with open(val_annotation, "r") as f:
    val_data = f.readlines()
train_data = [json.loads(x) for x in train_data]
val_data = [json.loads(x) for x in val_data]
images_path = "val/images"
labels_path = "val/labels"

In [17]:
for annotation in val_data:
    image_id = annotation["ID"]  # Extracting image name without extension
    image_path = os.path.join(images_path, f'{annotation["ID"]}.jpg')
    # read the image and get the image dimensions
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    # Create YOLO format label file
    label_file_path = os.path.join(labels_path, f"{image_id}.txt")
    with open(label_file_path, "w") as label_file:
        for gtbox in annotation["gtboxes"]:
            if gtbox["tag"] == "person":
                # Convert YOLO format and write to the label file
                x, y, w, h = gtbox["hbox"]  # Assuming you want to use visible box
                x_center = x + w / 2
                y_center = y + h / 2
                normalized_x_center = x_center / image_width
                normalized_y_center = y_center / image_height
                normalized_width = w / image_width
                normalized_height = h / image_height

                label_file.write(f"0 {normalized_x_center} {normalized_y_center} {normalized_width} {normalized_height}\n")

print("Conversion to YOLO format completed.")

Conversion to YOLO format completed.


In [13]:
test_image_path = "train/images/273271,1a27b000f0c7a077.jpg"
test_label_path = "train/labels/273271,1a27b000f0c7a077.txt"


import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def visualize_yolo(image_path, yolo_annotations_file):
    # Open the image
    img = Image.open(image_path)

    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Read YOLO annotations from file
    with open(yolo_annotations_file, 'r') as f:
        yolo_annotations = f.read().splitlines()

    # Parse YOLO annotations and draw bounding boxes
    for annotation in yolo_annotations:
        class_name, x_center, y_center, width, height = annotation.split()
        x_center, y_center, width, height = float(x_center), float(y_center), float(width), float(height)

        # Convert YOLO coordinates to image coordinates
        img_width, img_height = img.size
        xmin = int((x_center - width / 2) * img_width)
        ymin = int((y_center - height / 2) * img_height)
        xmax = int((x_center + width / 2) * img_width)
        ymax = int((y_center + height / 2) * img_height)

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)


    # Show the image with bounding boxes
    plt.show()
